# Playground for Transformers!

### Attention is all you need 
(https://arxiv.org/abs/1706.03762)

### For software vulnerability detection GYM

This is a minimal example of this **CRAZY** idea!

### Note

* LSTM is working with:
    * Adam Learning rate = 0.01
    * Overfitted on 14 epoch. 100% training accuracy.
    * Worked on 1K dataset sample
    * Using the `hidden`n or `cell` output from LSTM. Not the `output`.
    * Bidrectional (2 layers)
* Transformer:
    * Still trying to find the right combination
    * Adam Learning rate 0.0001

### Dataset

Import

In [1]:
import numpy as np
import pandas as pd
import os
import torch
import re
import torch.backends.cudnn as cudnn
import torchtext
import matplotlib.pyplot as plt
import time
from torch.autograd import Variable
from torch import nn, optim
from torch.optim import SGD,Adam
import torch.nn.functional as F
import random
from gensim.models.word2vec import Word2Vec
import torchtext.vocab as vocab

In [2]:
seed = 1234
torch.manual_seed(seed)
#device = torch.device("cpu");
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
cudnn.benchmark = True
cudnn.enabled = True
print(device)

cuda


Load playset dataset

In [2]:
dataset = pd.read_pickle('playset(0.25.2).pickle')

View dataset

In [3]:
dataset

functionSource  combine
93792  go_file_opener_open (GOFileOpener const *fo, g...    False
79770  updatePathMap(bool left_level) {\n\tPoint from...    False
66999  interpret_tilde(const char* path) {\n    stati...    False
44284  checkVarExp(\n        Absyn *node,\n        Ta...     True
49515  will_have_skip_worktree(const struct cache_ent...     True
...                                                  ...      ...
96701  AVLTree_insert(AVLTree * tree, void * data)\n{...    False
67815  remove_hook(const char *name, hookfn fn)\n{\n\...    False
88363  output_def(dico_stream_t str, struct gcide_db ...    False
65929  getState(\n\t\tFLMUINT\t\tuiFieldID)\n\t{\n\t\...    False
16426  untag_proplist(Pulse_Tag *tag, Eina_Hash **pro...     True

[100000 rows x 2 columns]

In [4]:
dataset.describe(include='all')

functionSource combine
count                                              100000  100000
unique                                             100000       2
top     v_identifier(void)\n#else\nv_identifier()\n#en...    True
freq                                                    1   50000

In [5]:
dataset.functionSource[1]

'checkCapture2() const\n{\n    for(int i=6;i<48;i++)\n    {\n        switch(board[i])\n        {\n        case MAN2:\n            if(board[i+5]==MAN1 || board[i+5]==KING1)\n                if(board[i+10]==FREE) return true;\n            if(board[i+6]==MAN1 || board[i+6]==KING1)\n                if(board[i+12]==FREE) return true;\n            break;\n        case KING2:\n            if(board[i-6]==MAN1 || board[i-6]==KING1)\n                if(board[i-12]==FREE) return true;\n            if(board[i-5]==MAN1 || board[i-5]==KING1)\n                if(board[i-10]==FREE) return true;\n            if(board[i+5]==MAN1 || board[i+5]==KING1)\n                if(board[i+10]==FREE) return true;\n            if(board[i+6]==MAN1 || board[i+6]==KING1)\n                if(board[i+12]==FREE) return true;\n        }\n    }\n    return false;\n}'

### Prepare data (in JSON)

#### Clean & Preprocessing

In [6]:
def clean(code):
    ## Remove newlines & tabs
    code = re.sub('(\n)|(\\\\n)|(\\\\)|(\\t)|(/)|(\\r)|(\")|(\')','',code)
    ## Remove code comments
    code = re.sub(r'/\*(.|[\r\n])*?\*/','',code)
    ## Mix split (characters and words)
    splitter = ' +|(;)|(\()|(==)|(\))|(=)|(\+)|(\-)|(\[)|(\])|(<)|(>)|({)|(#)'
    code = re.split(splitter,code)
    ## Remove None type
    code = list(filter(None, code))
    code = list(filter(str.strip, code))
    code = " ".join(code)
    ## Return list of tokens
    return(code)

In [7]:
## Clean the codes
dataset.functionSource = dataset.functionSource.apply(clean)

In [8]:
## Change labels boolean to 1 and 0
dataset.iloc[:,1] = np.multiply(dataset.iloc[:,1],1)

In [9]:
## Change column name
dataset = dataset.rename(columns={'functionSource':'codes', 'combine':'label'})

In [10]:
dataset.codes[24492]

'CardPowerOff ( reader* globalData, char socket ) { char cmd [ 4 ] , ack ; int retVal, actual, retryTimes = 2 ; # ifdef ASE_DEBUG syslog ( LOG_INFO, CardPowerOff - Enter ) ; # endif if ( ( retVal = cardCommandInit ( globalData, socket, 1 ) ) ) return retVal ; cmd [ 0 ] = ASE_PACKET_TYPE ( 0x50, globalData - > commandCounter, socket ) ; globalData - > commandCounter + + ; globalData - > commandCounter % = 4 ; cmd [ 1 ] = 0x21 ; cmd [ 2 ] = 0x0 ; cmd [ 3 ] = cmd [ 0 ] ^ cmd [ 1 ] ^ cmd [ 2 ] ; do { lock_mutex ( globalData ) ; retVal = sendControlCommand ( globalData, socket, cmd, 4, &ack, &actual, 0 ) ; unlock_mutex ( globalData ) ; retryTimes - - ; } while ( retVal ! = ASE_OK && retryTimes ) ; if during the 3 tries the command failed, return an error status if ( retVal < 0 ) { return retVal ; } if ( ack ! = 0x20 ) { return parseStatus ( ack ) ; } * if the card is present, change the status to powered off * if ( globalData - > cards [ ( int ) socket ] .status ) globalData - > cards [ ( i

In [11]:
false = dataset[dataset.iloc[:,1]==0]
true = dataset[dataset.iloc[:,1]==1]

In [12]:
## Split to train,test,valid
train = false[0:400].append(true[0:400])
test  = false[400:400].append(true[400:450])
valid = false[450:500].append(true[400:500])

## Shuffle
train = train.sample(frac=1).reset_index(drop=True)
test = test.sample(frac=1).reset_index(drop=True)
valid = valid.sample(frac=1).reset_index(drop=True)

In [ ]:
## Save to json
train.to_json('.data/train_.json', orient='records',lines=True)
test.to_json('.data/test_1k.json', orient='records',lines=True)
valid.to_json('.data/valid_1k.json', orient='records',lines=True)

In [3]:
## Define the field

CODES = torchtext.data.Field(batch_first=True)
LABEL = torchtext.data.LabelField(dtype=torch.int64)
fields = {'codes': ('codes', CODES), 'label': ('label', LABEL)}

In [4]:
## Import the 1K data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_1k.json',
                                        validation = 'valid_1k.json',
                                        test = 'test_1k.json',
                                        format = 'json',
                                        fields = fields)

In [4]:
## Import the min(10k) data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_min.json',
                                        validation = 'valid_min.json',
                                        test = 'test_min.json',
                                        format = 'json',
                                        fields = fields)

In [ ]:
## Import the ALL data as TabularDataset
train_data, valid_data, test_data = torchtext.data.TabularDataset.splits(
                                        path = '.data',
                                        train = 'train_all.json',
                                        validation = 'valid_all.json',
                                        test = 'test_all.json',
                                        format = 'json',
                                        fields = fields)

In [6]:
## Testing
print(vars(valid_data[0]))

{'codes': ['ap2sta_data_frame', '(', 'struct', '_adapter', '*adapter,', 'union', 'recv_frame', '*precv_frame,', 'struct', 'sta_info', '**psta', ')', '{', 'u8', '*ptr', '=', 'precv_frame', '-', '>', 'u.hdr.rx_data', ';', 'struct', 'rx_pkt_attrib', '*pattrib', '=', '&precv_frame', '-', '>', 'u.hdr.attrib', ';', 'structsta_priv', '*pstapriv', '=', '&adapter', '-', '>', 'stapriv', ';', 'structmlme_priv', '*pmlmepriv', '=', '&adapter', '-', '>', 'mlmepriv', ';', 'u8', '*mybssid', '=', 'get_bssid', '(', 'pmlmepriv', ')', ';', 'u8', '*myhwaddr', '=', 'myid', '(', '&adapter', '-', '>', 'eeprompriv', ')', ';', 'sint', 'bmcast', '=', 'IS_MCAST', '(', 'pattrib', '-', '>', 'dst', ')', ';', 'if', '(', 'check_fwstate', '(', 'pmlmepriv,', 'WIFI_STATION_STATE', ')', '&&', 'check_fwstate', '(', 'pmlmepriv,', '_FW_LINKED', ')', ')', '{', '*', 'if', 'NULL', '-', 'frame,', 'drop', 'packet', '*if', '(', '(', 'GetFrameSubType', '(', 'ptr', ')', ')', '==', 'WIFI_DATA_NULL', ')', 'return', '_FAIL', ';', '*', 

Doneeeeeeeeeeeeeeeee !

### Vocabulary-related preparation

In [5]:
##### Build the vocabulary

MAX_VOCAB_SIZE = 10000

CODES.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [6]:
print(f"Unique tokens in TEXT vocabulary: {len(CODES.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 10002
Unique tokens in LABEL vocabulary: 2


In [7]:
## Most common word
print(CODES.vocab.freqs.most_common(100))

[('(', 137341), (')', 137335), (';', 126428), ('-', 63469), ('=', 62779), ('>', 49012), ('{', 30774), ('*', 26072), ('if', 25875), ('}', 19709), ('+', 18237), ('0', 18226), ('[', 16280), (']', 16266), ('return', 13210), ('i', 10592), ('==', 10370), ('1', 10307), ('<', 9378), ('int', 8585), ('NULL', 7709), ('char', 7436), ('the', 7378), (',', 6696), ('!', 6244), ('struct', 5460), ('for', 4792), ('else', 4561), ('to', 4177), ('const', 4168), ('&&', 3850), ('#', 3560), ('sizeof', 3116), ('break', 3016), ('case', 3007), ('a', 2886), ('ret', 2631), ('||', 2556), ('len', 2522), ('unsigned', 2479), ('&', 2318), ('is', 2286), ('of', 2264), ('p', 2133), ('2', 2122), ('data', 2041), ('in', 1984), ('this', 1952), ('buf', 1898), ('0,', 1882), (':', 1781), ('size', 1735), ('n', 1662), ('while', 1641), ('s', 1604), ('void', 1590), ('%s', 1582), ('and', 1565), ('we', 1551), ('not', 1550), ('name', 1515), ('%s,', 1485), ('goto', 1471), ('j', 1462), ('}}', 1408), ('file', 1402), ('c', 1398), ('error', 

In [8]:
print(CODES.vocab.itos[:10])
print(LABEL.vocab.stoi)
print(CODES.vocab.stoi[CODES.pad_token])

['<unk>', '<pad>', '(', ')', ';', '-', '=', '>', '{', '*']
defaultdict(None, {0: 0, 1: 1})
1


In [9]:
## place into iterators
train_iterator, valid_iterator, test_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = 64,
    sort = False,
    device = device)

### Prepare Word2Vec (Optional)

In [ ]:
corpus = pd.read_json('.data/train.json',orient='records',lines=True)

w2v = Word2Vec(corpus.codes, size=300, workers=16, sg=1, min_count=3)
w2v.save('.data/node_w2v_128')

In [10]:
w2v = Word2Vec.load('.data/node_w2v_128')
w2v.wv.vectors

array([[ 0.04268723, -0.01990928, -0.10372822, ...,  0.34928635,
        -0.24622028, -0.02363101],
       [ 0.06655177, -0.08706249, -0.11346684, ...,  0.2967248 ,
        -0.16500187, -0.10260527],
       [ 0.10059763, -0.0993171 , -0.14234892, ...,  0.3913037 ,
        -0.22237949,  0.02339004],
       ...,
       [-0.0031671 ,  0.01939397, -0.00094254, ..., -0.06062187,
        -0.0873417 ,  0.10190531],
       [-0.03719744,  0.02801778,  0.02174594, ..., -0.05577604,
        -0.07265704, -0.00079473],
       [ 0.01092949, -0.03061507, -0.045645  , ..., -0.06999503,
        -0.15679213,  0.11291362]], dtype=float32)

### Transformer class (with LSTM)

In [10]:
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer,self).__init__()
        
        self.embed = nn.Embedding(num_embeddings=10002,
                                  embedding_dim=400)
        self.encode_layer = nn.TransformerEncoderLayer(d_model=400,
                                                       nhead=8,
                                                       dim_feedforward=2048,
                                                       dropout=0.1,
                                                       activation='relu')
        self.trans_encoder = nn.TransformerEncoder(self.encode_layer,
                                                   num_layers=1)
        self.lstm1 = nn.LSTM(input_size=400,
                            hidden_size=128,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(128*2,2)
    
    def forward(self,x):
        x = self.embed(x)
        x = self.trans_encoder(x)
        output, (hidden, cell) = self.lstm1(x)
        x = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        x = self.fc1(x)
        return(x)

model = Transformer()
model.to(device)
print(model)

Transformer(
  (embed): Embedding(10002, 400)
  (encode_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): Linear(in_features=400, out_features=400, bias=True)
    )
    (linear1): Linear(in_features=400, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=400, bias=True)
    (norm1): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((400,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (trans_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): Linear(in_features=400, out_features=400, bias=True)
        )
        (linear1): Linear(in_features=400, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_featur

### LSTM class

In [17]:
global k
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM,self).__init__()

        self.embed = nn.Embedding(num_embeddings=10002,
                                  embedding_dim=150)
        global k
        k = self.embed
        
        self.lstm1 = nn.LSTM(input_size=150,
                            hidden_size=64,
                            num_layers=2,
                            batch_first=True,
                            bidirectional=True)
        #self.fc1 = nn.Linear(128,64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64*2,2)
        
    
    def forward(self,x):
        x = self.embed(x)
        output, (hidden, cell) = self.lstm1(x)
        #x = output[:,-1,:]
        x = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        #x =self.dropout(torch.cat((cell[-2,:,:], cell[-1,:,:]), dim=1))
        #x = self.fc1(x)
        #x = F.relu(x)
        x = self.fc2(x)
        #x = F.log_softmax(x,dim=1)
        return(x)
    
model = LSTM()
model.to(device)
print(model)

LSTM(
  (embed): Embedding(10002, 150)
  (lstm1): LSTM(150, 64, num_layers=2, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)


### CNN Class

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.embed = nn.Embedding(num_embeddings=10002,
                                  embedding_dim=128)
        self.cnn1 = nn.Conv2d(in_channels=1,
                              out_channels=20,
                              kernel_size=2)
        self.cnn2 = nn.Conv2d(in_channels=1,
                              out_channels=20,
                              kernel_size=3)
        self.cnn3 = nn.Conv2d(in_channels=1,
                              out_channels=20,
                              kernel_size=4)
        self.fc1 = nn.Linear(60,2)
        #self.fc2 = nn.Linear(30,2)
    
    def forward(self,x):
        
        x = self.embed(x)
        x = x.unsqueeze(1)
        
        x1 = F.relu(self.cnn1(x))
        x1 = F.max_pool2d(x1,2)
        x1 = torch.flatten(x1,start_dim=1)
        
        x2 = F.relu(self.cnn2(x))
        x2 = F.max_pool2d(x2,3)
        x2 = torch.flatten(x2,start_dim=1)
        
        x3 = F.relu(self.cnn3(x))
        x3 = F.max_pool2d(x3,4)
        x3 = torch.flatten(x3,start_dim=1)
        
        x = torch.cat((x1,x2,x3), dim=1)
        self.fc1 = nn.Linear(x.shape[1],2)
        x = self.fc1(x.to(device))
        return(x)
    
model = CNN()
model.to(device)
print(model)

CNN(
  (embed): Embedding(10002, 128)
  (cnn1): Conv2d(1, 20, kernel_size=(2, 2), stride=(1, 1))
  (cnn2): Conv2d(1, 20, kernel_size=(3, 3), stride=(1, 1))
  (cnn3): Conv2d(1, 20, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=60, out_features=2, bias=True)
)


### Load Word2Vec weights to embeddings

In [14]:
weights = torch.FloatTensor(w2v.wv.vectors)
weights = weights.to(device)
model.embed = model.embed.from_pretrained(weights)
#model.embed = model.embed.weight.data.copy_(weights)

### Parameters

In [11]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,507,394 trainable parameters


### Prepare accuracy function

In [12]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [13]:
def softmax_accuracy(probs,all_labels):
    def getClass(x):
        return(x.index(max(x)))
    
    all_labels = all_labels.tolist()
    probs = pd.Series(probs.tolist())
    all_predicted = probs.apply(getClass)
    all_predicted.reset_index(drop=True, inplace=True)
    vc = pd.value_counts(all_predicted == all_labels)
    try:
        acc = vc[1]/len(all_labels)
    except:
        if(vc.index[0]==False):
            acc = 0
        else:
            acc = 1
    return(acc)

### Loss functions

In [14]:
## Define optimizer
#optimizer = SGD(model.parameters(), lr = 0.01)
optimizer = Adam(model.parameters(), lr=0.0001)

## Define loss function
#criterion = nn.BCELoss().to(device) ## Sigmoid activation function
#criterion = nn.NLLLoss().to(device) ### Log_softmax activation
criterion = nn.CrossEntropyLoss().to(device) ## No activation function bcs softmax included

### Training

In [15]:
epochs=100
for e in range(epochs):
    running_acc = 0
    running_loss = 0
    timer = time.time()
    model.train()

    for batch in train_iterator:
        optimizer.zero_grad()
        output = model(batch.codes)
        loss = criterion(output, batch.label)
        loss.backward()
        optimizer.step()
        acc = softmax_accuracy(output,batch.label)
        running_acc += acc.item()
        running_loss += loss.item()
    else:
        with torch.no_grad():
            model.eval()
            running_acc_val = 0
            running_loss_val = 0
            for batch in valid_iterator:
                output_val = model(batch.codes)
                loss_val = criterion(output_val,batch.label)
                acc_val = softmax_accuracy(output_val,batch.label)
                running_acc_val += acc_val.item()
                running_loss_val += loss_val.item()
        
        print("Epoch {} - Training acc: {:.6f} -Training loss: {:.6f} - Val acc: {:.6f} - Val loss: {:.6f} - Time: {:.4f}s".format(e+1, running_acc/len(train_iterator), running_loss/len(train_iterator), running_acc_val/len(valid_iterator), running_loss_val/len(valid_iterator), (time.time()-timer)))

Epoch 1 - Training acc: 0.601875 -Training loss: 0.658849 - Val acc: 0.702930 - Val loss: 0.588059 - Time: 133.0646s
Epoch 2 - Training acc: 0.682250 -Training loss: 0.600370 - Val acc: 0.734961 - Val loss: 0.559920 - Time: 147.2044s
Epoch 3 - Training acc: 0.719125 -Training loss: 0.561072 - Val acc: 0.723242 - Val loss: 0.564683 - Time: 144.9974s
Epoch 4 - Training acc: 0.729375 -Training loss: 0.545416 - Val acc: 0.738477 - Val loss: 0.534211 - Time: 147.8693s
Epoch 5 - Training acc: 0.751750 -Training loss: 0.516850 - Val acc: 0.747656 - Val loss: 0.514336 - Time: 146.3242s
Epoch 6 - Training acc: 0.780500 -Training loss: 0.483858 - Val acc: 0.750000 - Val loss: 0.561842 - Time: 149.5559s
Epoch 7 - Training acc: 0.795875 -Training loss: 0.463784 - Val acc: 0.764453 - Val loss: 0.506208 - Time: 146.0353s
Epoch 8 - Training acc: 0.811625 -Training loss: 0.441723 - Val acc: 0.737695 - Val loss: 0.533636 - Time: 144.8598s
Epoch 9 - Training acc: 0.821500 -Training loss: 0.417834 - Val 

KeyboardInterrupt: 

In [23]:
import sklearn.metrics
from sklearn.metrics import confusion_matrix

model.eval()
with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in train_iterator:
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = softmax_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += output_test.tolist()
        all_labels += batch.label.tolist()


print('Train acc: ',running_acc_test/len(train_iterator))
print('Train loss: ',running_loss_test/len(train_iterator))


def getClass(x):
    return(x.index(max(x)))

probs = pd.Series(all_pred)
all_predicted = probs.apply(getClass)
all_predicted.reset_index(drop=True, inplace=True)
vc = pd.value_counts(all_predicted == all_labels)

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_predicted)
print('Confusion matrix: \n',confusion)

Train acc:  0.934125
Train loss:  0.19550545418262483
Confusion matrix: 
 [[3718  282]
 [ 245 3755]]


### Testing

In [22]:
### SOFTMAX

import sklearn.metrics
from sklearn.metrics import confusion_matrix

model.eval()
with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in test_iterator:
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = softmax_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += output_test.tolist()
        all_labels += batch.label.tolist()


print('Test acc: ',running_acc_test/len(test_iterator))
print('Test loss: ',running_loss_test/len(test_iterator))


def getClass(x):
    return(x.index(max(x)))

probs = pd.Series(all_pred)
all_predicted = probs.apply(getClass)
all_predicted.reset_index(drop=True, inplace=True)
vc = pd.value_counts(all_predicted == all_labels)

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_predicted)
print('Confusion matrix: \n',confusion)


tn, fp, fn, tp = confusion.ravel()
print('\nTP:',tp)
print('FP:',fp)
print('TN:',tn)
print('FN:',fn)

## Performance measure
print('\nAccuracy: '+ str(sklearn.metrics.accuracy_score(y_true=all_labels, y_pred=all_predicted)))
print('Precision: '+ str(sklearn.metrics.precision_score(y_true=all_labels, y_pred=all_predicted)))
print('Recall: '+ str(sklearn.metrics.recall_score(y_true=all_labels, y_pred=all_predicted)))
print('F-measure: '+ str(sklearn.metrics.f1_score(y_true=all_labels, y_pred=all_predicted)))
#print('Precision-Recall AUC: '+ str(sklearn.metrics.average_precision_score(y_true=all_labels, y_score=all_pred)))
#print('AUC: '+ str(sklearn.metrics.roc_auc_score(y_true=all_labels, y_score=all_pred)))
print('MCC: '+ str(sklearn.metrics.matthews_corrcoef(y_true=all_labels, y_pred=all_predicted)))

Test acc:  0.714453125
Test loss:  0.8120712712407112
Confusion matrix: 
 [[362 138]
 [149 351]]

TP: 351
FP: 138
TN: 362
FN: 149

Accuracy: 0.713
Precision: 0.7177914110429447
Recall: 0.702
F-measure: 0.7098078867542973
MCC: 0.4261031294374961


In [ ]:
#### BINARY

import sklearn.metrics
from sklearn.metrics import confusion_matrix

with torch.no_grad():
    running_acc_test = 0
    running_loss_test = 0
    all_pred=[]
    all_labels=[]
    for batch in test_iterator:
        batch.codes = batch.codes.to(device)
        batch.label = batch.label.to(device)
        output_test = model(batch.codes).squeeze(1)
        loss_test = criterion(output_test,batch.label)
        acc_test = binary_accuracy(output_test,batch.label)
        running_acc_test += acc_test.item()
        running_loss_test += loss_test.item()
        all_pred += torch.round(output_test).tolist()
        all_labels += batch.label.tolist()


print('Test acc: ',running_acc_test/len(test_iterator))
print('Test loss: ',running_loss_test/len(test_iterator))

confusion = sklearn.metrics.confusion_matrix(y_true=all_labels, y_pred=all_pred)
print('Confusion matrix: \n',confusion)

# RESULTS